# Go/No-Go Task Minimal Preprocessing Pipeline

This notebook demonstrates how to use the minimal preprocessing pipeline for Go/No-Go task data. The minimal pipeline includes basic preprocessing steps that are essential for further analysis without more complex operations like ICA.

## Pipeline Overview

The minimal preprocessing pipeline includes the following steps:
1. **Load Data**: Load raw EEG data files
2. **Channel Preparation**: Configure channel types and handle missing montage positions
3. **Filtering**: Apply bandpass and notch filters to remove noise
4. **Epoching**: Segment the continuous data into epochs around stimulus events
5. **Save Checkpoint**: Save the preprocessed data for further analysis

## Setting Up the Environment

First, let's import the necessary libraries and configure the environment.

In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import numpy as np
import matplotlib.pyplot as plt
import mne
import yaml
from pathlib import Path
import logging

# Add the parent directory to path to import custom modules
sys.path.append(os.path.abspath('../..'))
# from custom_steps import CustomPrepChannelsStep, CustomEpochingStep

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger('gonogo_minimal_preprocessing')

# MNE settings
mne.set_log_level('INFO')
%matplotlib inline

## Loading the Pipeline Configuration

We'll load the minimal pipeline configuration from the YAML file.

In [2]:

def create_minimal_config():
    """
    Create a minimal pipeline configuration that includes:
    - Loading data
    - PreChannels step
    - Filtering
    - Epoching specific to Go/No-Go task
    """
    config = {
        "directory": {
            "root": "D:/Yann/neurotheque_resources/",
            "raw_data_dir": "data/pilot_data/tasks",
            "processed_dir": "data/processed",
            "reports_dir": "reports/minimal",
            "derivatives_dir": "derivatives"
        },
        "default_subject": "01",
        "default_session": "001",
        "default_run": "01",
        "pipeline_mode": "standard",
        "auto_save": True,
        "file_path_pattern": "sub-01_ses-001_task-gng_image_run-01_raw.fif",
        "pipeline": {
            "steps": [
                {
                    "name": "LoadData",
                    "params": {
                        "input_file": None  # Will be set by the pipeline based on file_path_pattern
                    }
                },
                {
                    "name": "PrepChannelsStep",
                    "params": {
                        "channel_types": {"X1:ECG": "ecg"},
                        "on_missing": "ignore"  # Handle missing channel positions
                    }
                },
                {
                    "name": "FilterStep",
                    "params": {
                        "l_freq": 1.0,
                        "h_freq": 40.0,
                        "notch_freqs": [50, 60]
                    }
                },
                {
                    "name": "EpochingStep",
                    "params": {
                        "tmin": -0.2,
                        "tmax": 1.0,
                        "baseline": [None, 0],  # Fixed None value for proper YAML serialization
                        "event_id": {"go": 1, "nogo": 2},  # Adjust based on your actual triggers
                        "stim_channel": "Trigger"  # Use the correct trigger channel name
                    }
                },
                {
                    "name": "SaveCheckpoint",
                    "params": {
                        "overwrite": True
                    }
                }
            ]
        }
    }
    
    return config


# Load the minimal pipeline configuration
try:
    # Try to load from existing YAML file first
    config_path = '../../configs/gonogo_minimal_pipeline.yml'
    with open(config_path, 'r') as f:
        config = yaml.safe_load(f)
    print(f"Loaded configuration from {config_path}")
except (FileNotFoundError, IOError):
    # If file doesn't exist, create the configuration programmatically
    print("Configuration file not found. Creating configuration programmatically.")
    config = create_minimal_config()

print("Pipeline steps:")
for step in config['pipeline']['steps']:
    print(f"- {step['name']}")

# Extract key configuration parameters
root_dir = config['directory']['root']
raw_data_dir = os.path.join(root_dir, config['directory']['raw_data_dir'])
processed_dir = os.path.join(root_dir, config['directory']['processed_dir'])
file_pattern = config['file_path_pattern']

# Ensure directories exist
os.makedirs(processed_dir, exist_ok=True)

Loaded configuration from ../../configs/gonogo_minimal_pipeline.yml
Pipeline steps:
- LoadData
- CustomPrepChannelsStep
- FilterStep
- CustomEpochingStep
- SaveCheckpoint


## Step 1: Loading the Data

The first step is to load the raw EEG data. For Go/No-Go tasks, we expect the data to be in FIFF format.

In [3]:
# Define the file path using the pattern from the configuration
file_path = os.path.join(raw_data_dir, file_pattern)

# Load the raw data
logger.info(f"Loading data from {file_path}")
raw = mne.io.read_raw_fif(file_path, preload=True)

# Display basic information about the raw data
print("\nRaw data information:")
print(f"Number of channels: {len(raw.ch_names)}")
print(f"Sampling frequency: {raw.info['sfreq']} Hz")
print(f"Duration: {raw.times[-1]:.2f} seconds")

# Plot the raw data (first 10 seconds)
raw.plot(duration=10, n_channels=20, title='Raw EEG data')

2025-04-13 01:37:06,386 - gonogo_minimal_preprocessing - INFO - Loading data from D:/Yann/neurotheque_resources/data/pilot_data/tasks\sub-01_ses-001_task-gng_image_run-01_raw.fif


Opening raw data file D:/Yann/neurotheque_resources/data/pilot_data/tasks\sub-01_ses-001_task-gng_image_run-01_raw.fif...
Isotrak not found
    Range : 81028 ... 385484 =    270.093 ...  1284.947 secs
Ready.
Reading 0 ... 304456  =      0.000 ...  1014.853 secs...

Raw data information:
Number of channels: 27
Sampling frequency: 300.0 Hz
Duration: 1014.85 seconds


2025-04-13 01:37:08,969 - OpenGL.acceleratesupport - INFO - No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate'


Using qt as 2D backend.


## Step 2: Channel Preparation

The `CustomPrepChannelsStep` properly handles the ECG channel and missing montage positions. This step is crucial for ensuring that all channels are correctly typed and named.

In [4]:
raw.ch_names


['EEG P3-Pz',
 'EEG C3-Pz',
 'EEG F3-Pz',
 'EEG Fz-Pz',
 'EEG F4-Pz',
 'EEG C4-Pz',
 'EEG P4-Pz',
 'EEG Cz-Pz',
 'CM',
 'EEG A1-Pz',
 'EEG Fp1-Pz',
 'EEG Fp2-Pz',
 'EEG T3-Pz',
 'EEG T5-Pz',
 'EEG O1-Pz',
 'EEG O2-Pz',
 'EEG X3:-Pz',
 'EEG X2:-Pz',
 'EEG F7-Pz',
 'EEG F8-Pz',
 'EEG X1:ECG-Pz',
 'EEG A2-Pz',
 'EEG T6-Pz',
 'EEG T4-Pz',
 'Pz',
 'Trigger',
 'Event']

In [19]:
# Extract parameters from the configuration
prep_channels_config = next(step for step in config['pipeline']['steps'] if step['name'] == 'PrepChannelsStep' or step['name'] == 'CustomPrepChannelsStep')

# Set the referencing parameters for Pz
if 'params' not in prep_channels_config:
    prep_channels_config['params'] = {}
    
prep_channels_config['params']['reference'] = {
    'method': 'average',  # Change from Pz (channels) to average reference
    # Uncomment these lines if you want to use specific channels instead
    # 'method': 'channels',
    # 'channels': ['Pz'],  # Our system already uses Pz as reference
    'projection': False
}

# Use the PrepChannelsStep with referencing
from scr.steps.prepchannels import PrepChannelsStep

# Create and apply the channel preparation step
logger.info(f"Preparing channels with referencing")
prep_channels_step = PrepChannelsStep(params=prep_channels_config['params'])
raw = prep_channels_step.run(raw)

# Continue with filtering using the FilterStep
from scr.steps.filter import FilterStep

# Extract filtering parameters from the configuration
filter_config = next(step for step in config['pipeline']['steps'] if step['name'] == 'FilterStep')

# Create and apply the filter step
logger.info(f"Applying filtering using FilterStep")
filter_step = FilterStep(params=filter_config['params'])
raw = filter_step.run(raw)

# Plot power spectral density after filtering
raw.plot_psd(fmax=filter_config['params']['h_freq']*1.5)

# Plot filtered data
raw.plot(duration=10, n_channels=20, title='Filtered EEG data')

StopIteration: 

## Step 3: Filtering

Filtering is a crucial step to remove unwanted frequency components from the signal. We apply:
1. A bandpass filter to remove slow drifts and high-frequency noise
2. Notch filters to remove power line interference

In [18]:
import matplotlib
matplotlib.use('Qt5Agg')  # or 'Qt5Agg', etc.
import matplotlib.pyplot as plt

# Extract filtering parameters from the configuration
filter_config = next(step for step in config['pipeline']['steps'] if step['name'] == 'FilterStep')

# Use the FilterStep directly instead of manual filtering
from scr.steps.filter import FilterStep

# Create and apply the filter step
logger.info(f"Applying filtering using FilterStep")
filter_step = FilterStep(params=filter_config['params'])
raw = filter_step.run(raw)

# Plot power spectral density after filtering
fig, ax = plt.subplots(figsize=(10, 6))
raw.compute_psd(fmax=filter_config['params']['h_freq']*1.5, picks='eeg').plot()
ax.set_title('Power Spectral Density after Filtering')
ax.set_xlabel('Frequency (Hz)')
ax.set_ylabel('Power Spectral Density (dB)')
plt.show()

# Plot filtered data
raw.plot(duration=10, n_channels=20, title='Filtered EEG data')

2025-04-13 01:49:50,537 - gonogo_minimal_preprocessing - INFO - Applying filtering using FilterStep


Filtering raw data in 1 contiguous segment
Setting up band-pass filter from 1 - 40 Hz

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandpass filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower passband edge: 1.00
- Lower transition bandwidth: 1.00 Hz (-6 dB cutoff frequency: 0.50 Hz)
- Upper passband edge: 40.00 Hz
- Upper transition bandwidth: 10.00 Hz (-6 dB cutoff frequency: 45.00 Hz)
- Filter length: 991 samples (3.303 s)

Filtering raw data in 1 contiguous segment
Setting up band-stop filter

FIR filter parameters
---------------------
Designing a one-pass, zero-phase, non-causal bandstop filter:
- Windowed time-domain design (firwin) method
- Hamming window with 0.0194 passband ripple and 53 dB stopband attenuation
- Lower transition bandwidth: 0.50 Hz
- Upper transition bandwidth: 0.50 Hz
- Filter length: 1981 samples (6.603 s)



[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Effective window size : 6.827 (s)


[Parallel(n_jobs=1)]: Done  17 tasks      | elapsed:    0.1s


Plotting power spectral density (dB=True).


C:\Users\s\AppData\Local\Temp\ipykernel_21416\732450856.py:18: UserWarning: Zero value in spectrum for channel Pz
  raw.compute_psd(fmax=filter_config['params']['h_freq']*1.5, picks='eeg').plot()
C:\Users\s\AppData\Local\Temp\ipykernel_21416\732450856.py:18: UserWarning: Infinite value in PSD for channel Pz.
These channels might be dead.
  raw.compute_psd(fmax=filter_config['params']['h_freq']*1.5, picks='eeg').plot()


Channels marked as bad:
none


## Step 4: Epoching

Epoching segments the continuous data into time windows around stimulus events. For Go/No-Go tasks, we create epochs for:
- `go` events (event ID: 1): Stimuli requiring a response
- `nogo` events (event ID: 2): Stimuli requiring response inhibition

The `CustomEpochingStep` properly uses the specified stim_channel for finding events.

In [ ]:
# Extract epoching parameters from the configuration
epoch_config = next(step for step in config['pipeline']['steps'] if step['name'] == 'CustomEpochingStep')
tmin = epoch_config['params']['tmin']  # Start time relative to event (seconds)
tmax = epoch_config['params']['tmax']  # End time relative to event (seconds)
event_id = epoch_config['params']['event_id']  # Event IDs to use
stim_channel = epoch_config['params']['stim_channel']  # Stimulus channel name
baseline = epoch_config['params']['baseline']  # Baseline correction period

# Execute the custom epoching step
logger.info(f"Creating epochs from {tmin}s to {tmax}s around events")
epoch_step = CustomEpochingStep(
    tmin=tmin, tmax=tmax, event_id=event_id,
    stim_channel=stim_channel, baseline=baseline
)
epochs = epoch_step.run(raw)

# Display information about the epochs
print("\nEpochs information:")
print(epochs)

# Count the number of each event type
print("\nEvent counts:")
event_counts = {event_name: len(epochs[event_name]) for event_name in event_id.keys()}
for event_name, count in event_counts.items():
    print(f"- {event_name}: {count} epochs")

# Plot the average of each event type
for event_name in event_id.keys():
    fig = epochs[event_name].average().plot(spatial_colors=True, title=f'Average ERP for {event_name} events')
    
# Plot epochs for a few selected channels
selected_channels = ['Fz', 'Cz', 'Pz']
epochs.plot_image(picks=selected_channels, combine='mean', title='Epochs image plot')

## Comparing Go vs. No-Go Responses

One of the key analyses in Go/No-Go tasks is comparing the ERPs for Go and NoGo conditions. Let's create a visualization to compare these responses.

In [ ]:
# Calculate ERPs for Go and NoGo conditions
go_erp = epochs['go'].average()
nogo_erp = epochs['nogo'].average()

# Create a comparison plot for key midline electrodes (Fz, Cz, Pz)
channels_to_plot = ['Fz', 'Cz', 'Pz']

fig, axes = plt.subplots(len(channels_to_plot), 1, figsize=(10, 12))
times = epochs.times * 1000  # Convert to milliseconds

for i, channel in enumerate(channels_to_plot):
    axes[i].plot(times, go_erp.data[go_erp.ch_names.index(channel)], 'b-', linewidth=2, label='Go')
    axes[i].plot(times, nogo_erp.data[nogo_erp.ch_names.index(channel)], 'r-', linewidth=2, label='NoGo')
    axes[i].axvline(x=0, color='k', linestyle='--')  # Mark stimulus onset
    axes[i].axhline(y=0, color='k', linestyle=':')
    axes[i].set_xlim([tmin*1000, tmax*1000])
    axes[i].set_title(f'Channel {channel}', fontsize=14)
    axes[i].set_ylabel('Amplitude (μV)')
    axes[i].legend(loc='best')
    
    # Highlight potential components
    if channel == 'Fz' or channel == 'Cz':
        axes[i].axvspan(200, 350, alpha=0.2, color='g', label='N2 range')
    if channel == 'Cz' or channel == 'Pz':
        axes[i].axvspan(350, 500, alpha=0.2, color='y', label='P3 range')

axes[-1].set_xlabel('Time (ms)')
plt.tight_layout()
plt.suptitle('Comparison of Go vs. NoGo ERPs', fontsize=16)
plt.subplots_adjust(top=0.92)
plt.show()

## Topographic Maps

Topographic maps help visualize the distribution of ERP components across the scalp.

In [ ]:
# Create topographic maps for key time points
# N2 component (around 200-350 ms)
# P3 component (around 350-500 ms)

n2_time = 0.250  # 250 ms
p3_time = 0.400  # 400 ms

fig, axes = plt.subplots(2, 2, figsize=(10, 8))

# N2 Go
go_erp.plot_topomap(times=n2_time, axes=axes[0, 0], show=False, colorbar=True)
axes[0, 0].set_title('Go - N2 (250 ms)')

# N2 NoGo
nogo_erp.plot_topomap(times=n2_time, axes=axes[0, 1], show=False, colorbar=True)
axes[0, 1].set_title('NoGo - N2 (250 ms)')

# P3 Go
go_erp.plot_topomap(times=p3_time, axes=axes[1, 0], show=False, colorbar=True)
axes[1, 0].set_title('Go - P3 (400 ms)')

# P3 NoGo
nogo_erp.plot_topomap(times=p3_time, axes=axes[1, 1], show=False, colorbar=True)
axes[1, 1].set_title('NoGo - P3 (400 ms)')

plt.tight_layout()
plt.suptitle('Topographic Distribution of ERP Components', fontsize=16)
plt.subplots_adjust(top=0.9)
plt.show()

## Saving the Processed Data

Finally, we'll save the preprocessed epochs for future analysis.

In [ ]:
# Define output file path
sub_id = config['default_subject']
ses_id = config['default_session']
run_id = config['default_run']
output_dir = os.path.join(processed_dir, f"sub-{sub_id}", f"ses-{ses_id}")
os.makedirs(output_dir, exist_ok=True)

output_file = os.path.join(output_dir, f"sub-{sub_id}_ses-{ses_id}_task-gng_run-{run_id}_preprocessed-epo.fif")

# Save epochs
logger.info(f"Saving preprocessed epochs to {output_file}")
epochs.save(output_file, overwrite=True)

print(f"\nPreprocessed data saved to: {output_file}")

## Conclusion

This notebook demonstrated the minimal preprocessing pipeline for Go/No-Go task data. We've covered:

1. **Loading the raw data** from FIFF files
2. **Preparing channels** using the custom step that properly handles ECG channels and missing montage positions
3. **Filtering** with bandpass (1-40 Hz) and notch filters (50/60 Hz)
4. **Epoching** the data around Go and NoGo events with baseline correction
5. **Visualizing** the ERPs and their topographic distributions
6. **Saving** the preprocessed data for future analysis

The minimal pipeline provides the essential preprocessing steps for analyzing Go/No-Go task data without more complex operations like ICA or advanced artifact rejection. For more complex preprocessing needs, consider using the intermediate or advanced pipeline configurations.

### Key ERP Components in Go/No-Go Tasks

- **N2 component** (200-350 ms): Associated with conflict monitoring and inhibitory processing. Typically enhanced (more negative) for NoGo compared to Go trials, especially at frontocentral electrodes (Fz, Cz).
- **P3 component** (350-500 ms): Associated with response inhibition (NoGo-P3) and response execution (Go-P3). The NoGo-P3 typically has a more anterior distribution (Cz), while the Go-P3 has a more posterior distribution (Pz).

These components are key neural markers of response inhibition processes in the Go/No-Go task.